In [1]:
!pip install acceration
!pip install peft
!pip install transformers
!pip install bitsandBytes
!pip install datasets

ERROR: Could not find a version that satisfies the requirement acceration (from versions: none)
ERROR: No matching distribution found for acceration
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Foun

In [2]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=b95b8ab8e0f0e157f73fbff1292fceb965e5a7880c66c29925330439394e73c4
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built GPUtil


In [1]:
import torch
import GPUtil
import os

GPUtil.showUtilization()
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead")

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU is available


In [30]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from transformers import LlamaTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()

In [3]:
if "COLAB_GPU" in os.environ:
  !huggingface-cli login
else:
  notebook_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineG

In [4]:
base_model_id = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
import os

file_name = "BhagavadGita.txt"
file_path = os.path.abspath(file_name)

print("Full path:", file_path)


Full path: /content/BhagavadGita.txt


In [12]:
from datasets import load_dataset, Features, Value

features = Features({'text': Value('string')})

train_dataset = load_dataset(
    "text",
    data_files={"train": ["/content/data/BhagavadGita.txt", "/content/data/siva_puranam.txt"]},
    split="train",
    features=features
)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [18]:
train_dataset["text"][1000]

'beginning to have doubts about the capabilities of his army. He knows that'

In [20]:
tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

In [22]:
if tokenizer.pad is None:
  tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})

In [35]:
tokenized_train_dataset=[]
for phrase in train_dataset:
    tokenized_train_dataset.append(tokenizer(phrase["text"]))

In [36]:
tokenized_train_dataset[100]

{'input_ids': [1, 2225, 23059, 1426, 297, 7513, 322, 17551, 363, 8041, 29889, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [37]:
tokenizer.eos_token

'</s>'

In [38]:
model.gradient_checkpointing_enable()
model=prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [40]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Fix: Ensure tokenizer has a pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # or use tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Optional: Resize model embeddings if you've added a new pad token
# model.resize_token_embeddings(len(tokenizer))  # Only if you added new token

# Setup Trainer
trainer = Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    args=TrainingArguments(
        output_dir="./finetunedModel",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        num_train_epochs=5,
        learning_rate=1e-4,
        max_steps=100,
        fp16=False,
        optim="paged_adamw_8bit",  # Use standard "adamw_torch" if "paged_adamw_8bit" is unsupported
        logging_dir="./log",
        save_strategy="epoch",
        save_steps=50,
        logging_steps=20,
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
)

# Optional: Prevent warning for use_cache during training
model.config.use_cache = False

# Train the model
trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
20,4.030800
40,3.612900
60,3.304700
80,3.119700
100,3.187800


TrainOutput(global_step=100, training_loss=3.4511661529541016, metrics={'train_runtime': 382.6475, 'train_samples_per_second': 1.045, 'train_steps_per_second': 0.261, 'total_flos': 328371050692608.0, 'train_loss': 3.4511661529541016, 'epoch': 0.015875535799333228})

In [42]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, BitsAndBytesConfig
from peft import PeftModel

base_model_id = "meta-llama/Llama-2-7b-chat-hf"

nf4Config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=nf4Config, device_map="auto", trust_remote_code=True, use_auth_token=True)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [43]:
tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

modelFinetuned = PeftModel.from_pretrained(model, "finetunedModel/checkpoint-100")

/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.default.we

In [45]:
user_question = "what is mentioned in shiva puran about lord shiva?"

eval_prompt = f"Question: {user_question} just answer this question accurately and concisely. \n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
modelFinetuned.eval()

with torch.no_grad():
  print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens=1024)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Question: what is mentioned in shiva puran about lord shiva? just answer this question accurately and concisely. 
sorry for my bad english.

Answer: The Shiva Purana is a Hindu religious text that contains numerous stories, legends, and teachings about Lord Shiva. Here are some key aspects of Lord Shiva mentioned in the Shiva Purana:

1. Lord Shiva as the ultimate reality: The Shiva Purana describes Lord Shiva as the ultimate reality, beyond the dualities of creation and destruction, birth and death, and good and evil.
2. Shiva as the creator: According to the Shiva Purana, Lord Shiva is the creator of the universe, including the gods, humans, and other living beings.
3. Shiva as the destroyer: Lord Shiva is also described as the destroyer of evil and injustice, and the one who brings about transformation and change.
4. Shiva as the sustainer: Lord Shiva is seen as the sustainer of the universe, maintaining the balance and harmony of creation.
5. Shiva as the lord of yoga: The Shiva Pu

In [50]:
user_question = "what is the role of lord krishna in bhagavad gita?"

eval_prompt = f"Question: {user_question} just answer this question accurately and concisely. \n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
modelFinetuned.eval()

with torch.no_grad():
  print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens=1024)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Question: what is the role of lord krishna in bhagavad gita? just answer this question accurately and concisely. 
satya narayan goyal 27 February 2023
Lord Krishna plays a pivotal role in the Bhagavad Gita as a spiritual guide and mentor. He is the charioteer of Arjuna, and through his divine discourse, he teaches Arjuna the principles of yoga, selfless action, and the nature of the self. Krishna is the embodiment of the divine self, and his teachings in the Gita are meant to help Arjuna and all seekers of truth to realize their full potential and achieve spiritual liberation.

Some of the key roles of Lord Krishna in the Bhagavad Gita include:

1. Teacher: Lord Krishna teaches Arjuna the principles of yoga, including the yoga of action (karma yoga), the yoga of knowledge (jnana yoga), and the yoga of devotion (bhakti yoga).
2. Guide: Krishna acts as a spiritual guide, offering advice and guidance to Arjuna on how to navigate the challenges of life and achieve spiritual liberation.
3. 